In [34]:
import pandas as pd
import numpy as np
from instructions import load_df_from_pkl, save_df_as_pkl


data_directory_vsc = 'C:/Users/Marta/Desktop/Studia/CDV/IV semestr 2022L/Wykorzystanie Pythona w uczeniu maszynowym/ml_project/project/ML_PROJECT_2022/data/raw'
data_directory_colab = '/content/project/data'
data_directory = data_directory_vsc

In [6]:
X_std = load_df_from_pkl(data_directory, 'X_std')

###Interquartile Range Method

In [17]:
def removing_iqr(dataset: pd.DataFrame) -> pd.DataFrame:
    for column in dataset:
        q1, q3 = np.percentile(dataset[column], [25, 75])
        iqr = q3 - q1
        dataset[column] = (dataset[column] < (q1 - 1.5 * iqr)) | (dataset[column] > (q3 + 1.5 * iqr))
    return dataset

In [18]:
# removing outliers from a dataframe using IQR method
X_copy = X_std.copy()
X_outliers_rem = removing_iqr(X_copy)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3746,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,F

In [38]:
# counting outliers occurance
outliers_count = X_outliers_rem.sum(axis=1)

# percentage of the total
percentage = []
for i in outliers_count:
    percentage.append(i/10000) 

In [52]:
table = pd.DataFrame(percentage).sort_values(by=[0], ascending=False)
table

,0
3667,0.0105
2844,0.0099
2524,0.0098
2820,0.0097
3458,0.0096
...,...
3204,0.0048
7,0.0048
909,0.0047
3594,0.0046


In [29]:
X_nans = X_std.mask(X_outliers_rem == True, np.nan)

In [31]:
from sklearn.impute import KNNImputer

def replace_missing_values(dataset: pd.DataFrame, n_neighbors: int) -> pd.DataFrame:
    knn_imputer = KNNImputer(n_neighbors=n_neighbors)
    imputed_df = knn_imputer.fit_transform(dataset)
    return pd.DataFrame(imputed_df)

In [32]:
# replacing missing values
X_final = replace_missing_values(X_nans, 5)
X_final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.956603,-2.090995,-0.779319,-1.705003,-0.679182,-1.152735,-0.520916,0.328344,-0.407837,-2.204406,0.507280,0.552020,2.402776,-1.676888,0.180629,-1.240344,1.511586,-1.603184,-0.260460,-0.318621,0.867121,-0.014336,-0.391808,-1.014170,-0.597909,-1.080459,0.818631,-0.605869,0.705884,-0.114408,-1.457485,1.633805,0.017490,-0.458193,0.427064,0.637615,0.075962,0.275046,-0.071771,-0.365416,...,-0.053678,-1.097888,0.428001,-0.478771,0.620406,-0.407438,-0.199193,-0.218744,-0.924045,0.396945,1.446239,-0.314009,-0.397713,-0.964456,-0.666111,0.243591,0.336081,-0.099903,-0.585966,1.236565,0.481132,-0.595633,-1.422636,-2.584841,0.426585,-0.357412,0.436126,-0.007210,-0.240937,-0.062679,0.302007,-0.046534,-1.034740,0.476194,-0.595632,-2.131212,0.161407,0.975154,-0.299314,1.715553
1,0.081790,0.380063,-0.007431,-0.086273,1.757154,-0.864456,1.214323,0.293748,1.502145,0.643388,0.111160,1.313802,-0.853707,-0.526520,-0.343671,0.210823,0.814955,0.732632,0.984040,1.334054,-0.336630,-0.159911,-0.659259,-0.580516,0.061180,0.757203,0.735510,-1.908703,0.382054,0.157993,0.328743,-0.041166,0.742035,0.721931,-1.406621,0.262560,-0.271067,0.639786,0.940653,0.463612,...,-0.529203,0.229033,-1.371223,-1.737892,-2.019530,0.045241,-1.077180,-2.161109,-1.232478,0.333801,-0.174797,-0.493242,-1.175119,0.402351,-0.875833,1.994322,-1.088555,0.166724,0.227149,0.281282,-0.948418,-1.014714,0.649804,-0.802130,0.857594,0.258438,0.330319,-0.842376,-0.096595,0.166261,-0.707085,-0.820444,-2.130636,0.163723,-0.142053,-0.421798,-0.529819,-1.465157,0.426158,-1.340783
2,0.844093,-0.989109,0.092602,-0.173940,-1.606071,-1.482817,-0.591553,-0.570965,-1.099315,-0.744593,-0.011426,-1.396448,0.751181,0.236124,-1.489629,-0.842989,0.067894,0.297898,1.677366,-1.043738,-0.929305,0.411600,-0.431322,0.487998,1.050045,0.340709,-0.902760,0.857071,-0.967624,-0.149667,0.735180,-0.229023,0.297028,-1.449152,-0.166248,1.065325,-0.758463,-0.550049,-0.001391,0.079912,...,-0.956062,-0.050125,-0.676551,0.393267,-0.356788,-0.892507,0.487859,-0.883610,1.004461,-0.128329,0.271615,0.066370,-1.247462,0.306851,-0.560067,1.995250,-1.012588,1.191189,-0.590863,0.478444,-1.246916,-0.140078,-1.603945,-1.972204,-0.720366,0.413005,-0.351371,0.248412,-0.319977,0.912962,1.142062,-0.728438,0.453446,1.796409,-0.166745,-0.263702,-0.719131,-1.979327,1.082855,1.278390
3,0.487054,0.279889,1.110703,0.506412,-0.625261,1.221940,-0.314777,-0.955742,-1.326159,-0.196054,-0.900656,0.099878,0.474808,0.089506,0.441067,0.178789,-0.260107,-1.189386,-0.180825,-1.179266,-0.337156,1.869190,-0.524703,-0.049726,-0.448100,-0.306931,0.385809,-1.558231,-1.305089,0.048084,1.789095,0.330696,-0.278055,0.152771,-0.018828,-0.714429,-0.645852,0.697244,0.486368,-0.427457,...,-1.498174,-0.107230,0.971697,1.521597,-1.391103,0.753242,-2.544540,-1.367446,1.011627,-1.290280,-0.216710,2.030235,0.859280,1.087383,1.945553,-1.428002,1.902610,-1.702689,2.211995,-1.176164,-0.706983,-1.848721,-0.220121,0.047022,0.819059,1.406333,-0.870325,-0.521128,0.928824,-2.192273,-1.574800,0.695451,1.096150,0.211454,2.165549,0.227377,0.106670,-0.987100,0.916110,1.843469
4,0.196247,0.802160,-1.140956,0.029840,1.202957,-0.608846,-0.238682,-0.083731,-0.327290,-0.241897,1.058027,-0.651218,1.141423,-0.944296,0.311642,-0.837027,-0.553130,-0.678254,1.863799,0.053233,-0.611168,-0.855999,0.505475,-0.556214,1.880076,-1.063857,-0.506148,0.767029,0.429708,0.429846,0.032054,0.519737,-0.200399,-0.756747,-0.944204,-1.035587,0.509075,-1.070573,-1.106736,0.349676,...,-0.032635,0.545677,-2.116226,-0.187212,0.702448,0.549819,-0.558109,1.610469,0.117958,-1.361892,0.831100,-1.383278,-0.018114,-0.897597,1.837300,0.427783,0.003274,-2.321897,0.102558,0.860631,0.843404,1.218868,1.025027,-0

In [35]:
save_df_as_pkl(X_final, data_directory, 'X_outliers_removed')

Dataframe 'X_outliers_removed.pkl' saved as a pickle.
